# Make a model & Run it with OpenSEES




In [50]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [51]:
#Make a dictionary of nodes, mesh is 27x79 
#nodes on deck
NODES={}
for j in range(1,28):
    for i in range(1,80):
        n=100*j+i
        x=(i-1)*10
        y=(j-14)*11
        z=0
        NODES[n]=[x,y,z]

#nodes on girders
for j in (2,10,18,26):
    for i in range(1,80):
        n=10000+100*j+i
        x=(i-1)*10
        y=(j-14)*11
        z=-24
        NODES[n]=[x,y,z]
        
#nodes on BC
for j in (2,10,18,26):
    for i in (1,79):
        n=20000+100*j+i
        x=(i-1)*10
        y=(j-14)*11
        z=-48
        NODES[n]=[x,y,z]

In [52]:

#SHELL ELEMENTS FOR DECK
SHELLS={}
for j in range(1,27):
    for i in range(1,79):
        n1=100*j+i
        n2=100*(j+1)+i
        n3=100*(j+1)+(i+1)
        n4=100*j+(i+1)
        SHELLS[n1]=[n1,n2,n3,n4]



In [53]:
#BEAM ELEMENTS FOR GIRDERS
BEAMS={}
for j in (2,10,18,26):
    for i in range(1,79):
        n1=10000+100*j+i
        n2=10000+100*j+i+1
        BEAMS[n1]=[n1,n2]
        
#BEAM ELEMENTS FOR DIAPHRAGMS
for j in (2,10,18):
    for i in (1,20,40,60,79):
        n1=10000+100*j+i
        n2=10000+100*(j+8)+i
        BEAMS[n1+10000]=[n1,n2]


In [54]:
#LINKS
LINKS={}
for j in (2,10,18,26):
    for i in range(1,80):
        n1=100*j+i
        n2=10000+100*j+i
        LINKS[n1]=[n1,n2]


In [62]:
#plot the model
x=[NODES[n][0] for n in sorted(NODES.keys())]
y=[NODES[n][1] for n in sorted(NODES.keys())]
z=[NODES[n][2] for n in sorted(NODES.keys())]
node_trace = go.Scatter3d(x=x,y=y,z=z, mode='markers',marker=dict(size=3,color='blue'),name='Nodes')
beam_trace=[]

for b in sorted(BEAMS.keys()):
    if b==sorted(BEAMS.keys())[0]:
        showlegend=True
    else:
        showlegend=False
    beam_trace.append(go.Scatter3d(x=[NODES[BEAMS[b][0]][0],NODES[BEAMS[b][1]][0]],
                                   y=[NODES[BEAMS[b][0]][1],NODES[BEAMS[b][1]][1]],
                                   z=[NODES[BEAMS[b][0]][2],NODES[BEAMS[b][1]][2]],
                                   mode='lines',showlegend=showlegend,name='Beams',line=dict(color='red')))
link_trace=[]
for l in sorted(LINKS.keys()):
    if l==sorted(LINKS.keys())[0]:
        showlegend=True
    else:
        showlegend=False   
    link_trace.append(go.Scatter3d(x=[NODES[LINKS[l][0]][0],NODES[LINKS[l][1]][0]],
                                   y=[NODES[LINKS[l][0]][1],NODES[LINKS[l][1]][1]],
                                   z=[NODES[LINKS[l][0]][2],NODES[LINKS[l][1]][2]],
                                   mode='lines',showlegend=showlegend,name='Links',line=dict(color='black')))
noaxis=dict(showbackground=False,showline=False,zeroline=False,showgrid=False,showticklabels=False,title='')
layout=dict(title='Model',scene=dict(aspectmode='data',xaxis=noaxis,yaxis=noaxis,zaxis=noaxis))
fig = go.Figure(data=[node_trace]+beam_trace+link_trace,layout=layout)
py.iplot(fig)